# Goal: Generate tiles from full filters 

With Apeer, we annotated a full filter, selecting the tray and holder as background. In order to obtain annotated images on which we can train a neural network, we must first create these "tiles" and for this we use this notebook to manually visualize, cut and export full filter images. 

Generate cropped image without tray or holder visible
Generate images with the matching sizes automatically
Generate mirror, rotated, etc images (optional as augmentation is done after annotation)

Manual operations: remove x-y margin from the original image


In [ ]:
import re
import cv2
import PIL.Image
from PIL import *
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.ndimage
import skimage
import os

from PIL import Image, ImageOps

from IPython.display import display
from IPython.display import Markdown as md

from os import listdir
from os.path import isfile, join

# import ipywidgets.interact class, this class will represent a slide bar.
from ipywidgets import *

#from ipyfilechooser import FileChooser # Does not work remotely but is really cool https://pypi.org/project/ipyfilechooser/

In [ ]:
#input_dir = "./input_images/"
input_dir = "./input_images/complicated_filters/"
output_dir_cropped = "./output_images/cropped_filters_2"
output_dir_tiles = "./output_images/tiles_2"


In [ ]:
# Im is np.array: 
# tiles = [im[x:x+M,y:y+N] for x in range(0,im.shape[0],M) for y in range(0,im.shape[1],N)]

In [ ]:
# Non interactive argument: https://forum.image.sc/t/non-interactive-arguments-in-ipywidgets/32649
#@interact_manual(x=(0, 100))
#def blur(imarray = fixed(imarray), x = 5):
#    print(type(imarray))
#    blurred = skimage.filters.gaussian(imarray, sigma=(x, x))
#    plt.imshow(blurred)
    
# use interact decorator to decorate the function, so the function can receive the slide bar's value with parameter x.
#@interact(x=(0, 1000))
#def crop_top_margin(x, image_path):
#    print(f"The value of x is {x}")

# Preview Image and manually set margins to cut the tray and filter holder

Only select good quality filters. Filters with poor focus, clogged materials, etc are not the target of the tool for now. 

In [ ]:
# Preview
from IPython.display import Image



@interact
def show_images(file=os.listdir(input_dir)):
    #img = Image.open(input_dir + file)
    print(input_dir + file)
    preview_file_path = input_dir + file
    imarray = plt.imread(input_dir + file)
    display(PIL.Image.fromarray(imarray))
    
  

# Select the file from list for preprocessing

In [ ]:
import glob

import IPython.display
import ipywidgets

path = ipywidgets.Text(
    description='String:',
    value='./input_images/complicated_filters')

IPython.display.display(path)

options = glob.glob('{}/*'.format(path.value))

files = ipywidgets.SelectMultiple(
    description='Dataset(s)',
    options=options,
    layout=widgets.Layout(width='900px', height='200px')
)

IPython.display.display(files)

In [ ]:
# Show selected file (above)

# Takes first file is more than one
if len(files.value) != 0:
    image_path = files.value[0]
    print(image_path)
    imarray = plt.imread(image_path)
#    plt.imshow(imarray)
#    plt.show()

    # Opens a image in RGB mode
    #imarray = plt.imread("./input_images/whole_filters/Boiled_Clams_od_020322_uzorak_br_6-Uzorak_br_6_Light_II.tif")
    im = PIL.Image.fromarray(imarray)

    # Size of the image in pixels (size of original image)
    # (This is not mandatory)
    width, height = im.size
    print(im.size)

    # Setting the points for cropped image
    left = 400
    top = 200
    right = width - 200
    bottom = height - 200

    print((left, top, right, bottom))

    # Cropped image of above dimension
    # (It will not change original image)
    im1 = im.crop((left, top, right, bottom))

    # Shows the image in image viewer
    display(im1)



## If the above cut is satisfying, save the image

In [ ]:
# output_dir_cropped

output_image_path = os.path.join(output_dir_cropped, "cropped_" + os.path.basename(image_path))
print(f"Save image {image_path} into {output_image_path}")

# Uncomment to save
print(type(im1))
im1.save(output_image_path)

# 

# Sliding window within the cropped image: cut fixed size tiles

In [ ]:
# Efficient sliding window: https://stackoverflow.com/questions/61051120/sliding-window-on-a-python-image

target_width = 752
target_height = 480
step_size = 250



def sliding_window(image, stepSize, windowSize, idx):
    for y in range(0, image.shape[0]-target_height, stepSize):
        for x in range(0, image.shape[1]-target_width, stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])
            
            

def extractFeatures(window, idx):
    # avoid windows with size not matching
    #print(window.shape)
    if window.shape != (480, 752, 3):
        print(f"Ignore {idx}")
        return "Ignore window"
    else:
        # For each window, save the original, a flip and a mirror version (flip + mirror juged too much)
        im = PIL.Image.fromarray(window)
        im_path = os.path.join(output_dir_tiles, f"{idx[0]}_{idx[1]}_auto_generated_tile.jpg")
        im.save(im_path)

        #imflip_path = os.path.join(output_dir_tiles, f"{idx[0]}_{idx[1]}_auto_generated_tile_flip.jpg")
        #im_flip = ImageOps.flip(im)
        #im_flip.save(imflip_path)

        #immirror_path = os.path.join(output_dir_tiles, f"{idx[0]}_{idx[1]}_auto_generated_tile_mirror.jpg")
        #im_mirror = ImageOps.mirror(im)
        #im_mirror.save(immirror_path)

    return "features"

#images = []


In [ ]:


cropped_image_list = next(os.walk(output_dir_cropped), (None, None, []))[2]  # [] if no file



# SLIDE !
#cropped_image_list = [cropped_image_list[0]]
#print(cropped_image_list)

for i, image_name in enumerate(cropped_image_list):
    print(f"Image name: {image_name}")
    image_path = os.path.join(output_dir_cropped, image_name)
    print(f"Image path: {image_path}")
    print(f"Image ID: {i}")
    image = plt.imread(image_path)
    features = []
    windows = sliding_window(image, step_size, (target_width, target_height), i)
    for t, window in enumerate(windows):
        featureVector = extractFeatures(window[2], (i,t))
        features.append(featureVector)

    #numpy.savetxt('image_name_features.txt', feautres, delimiter=",")
